# Studying micrograd + Building it from scratch

# 4/30/2023

**I've added the following**

1. A hyperbolic tangent function (tanh) and its corresponding backward function for gradient computation.
2. A backpropagation method that computes gradients for all the nodes in the computational graph.
3. An example that demonstrates how to use the
 Value class to perform various operations and compute gradients using backpropagation.

In [1]:
from math import exp
import numpy as np

In [2]:
class Value:
    def __init__(self, data, _children=(), _op=""):
        self.data = data
        self.grad = 0.0

        self._backward = lambda: None
        self._prev = set(_children) # stores previous objects
        self._op = _op # stores the operation
    
    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})" # added grad
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '*')
    
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def tanh(self):
        x = self.data
        t = (exp(2*x) - 1)/(exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
    
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    def backward(self): # performs backpropagation to calculate gradients for all the nodes
    
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
    
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [3]:
# an example

x = Value(2.0)
y = Value(3.0)

x_2 = x * x
x_2_y = x_2 * y
xy = x * y
tanh_xy = xy.tanh()
# func
z = x_2_y + tanh_xy

# grads
z.backward()

# results
print(f"x: {x}")
print(f"y: {y}")
print(f"X_2: {x_2}")
print(f"x_2_y: {x_2_y}")
print(f"xy: {xy}")
print(f"tanh_xy: {tanh_xy}")
print(f"z: {z}")

x: Value(data=2.0, grad=12.000073729642216)
y: Value(data=3.0, grad=4.00004915309481)
X_2: Value(data=4.0, grad=3.0)
x_2_y: Value(data=12.0, grad=1.0)
xy: Value(data=6.0, grad=2.4576547405286142e-05)
tanh_xy: Value(data=0.9999877116507956, grad=1.0)
z: Value(data=12.999987711650796, grad=1.0)
